In [71]:
# Importing packages 
import mediapipe as mp
import numpy as np
import os
import uuid
import cv2

# necessary utilities
mp_drawing = mp.solutions.drawing_utils # for the nodes on our hands
mp_hands = mp.solutions.hands #for the hand model of mediapipe

In [72]:
def get_hand_label(index, hand, results):
    output = None
    for idx ,c in enumerate(results.multi_handedness):
        if(c.classification[0].index == index):
            # here we extracted necessary information of the correct hand with the help of index
            label  = c.classification[0].label
            score = round(c.classification[0].score,2)
            text = f"{label} ({score})"
            
            # using coordinates of the wrist
            coor = [hand.landmark[mp_hands.HandLandmark.WRIST].x, hand.landmark[mp_hands.HandLandmark.WRIST].y]

            # print(coor_to_print)
            output = text, coor
            
    return output

In [73]:
def print_joint_angle(image, results, joint_list):
    
    # Loop through hands
    for hand in results.multi_hand_landmarks:
        #Loop through joint sets 
        for joint in joint_list:
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) # First coord
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) # Second coord
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) # Third coord
            
            radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*180.0/np.pi)
            
            if angle > 180.0:
                angle = 360-angle
                
            # window dimesions
            dim = [image.shape[1],image.shape[0]]
                
            cv2.putText(image, str(round(angle, 2)), tuple(np.multiply(b, dim).astype(int)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    return image

In [74]:
# Hand detection
def print_hand_labels(image, num, hand, results):
    if(get_hand_label(num, hand, results)):
        text , coordinates = get_hand_label(num, hand, results)
        
        # window dimesions
        dim = [image.shape[1],image.shape[0]]
        
        # changing coordinates according to the window size
        coor_to_print = tuple(np.multiply(np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x, hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                    dim).astype(int))
        
        cv2.putText(image, text, coor_to_print, cv2.FONT_HERSHEY_SIMPLEX, 1, (1,0,0),2,cv2.LINE_AA)
        return image

In [75]:
#  Setting up the webcam
cap = cv2.VideoCapture(0)

# min_detection_condfidence for the accuracy on first time detection of our hand
# min_tracking_confidence for the accuracy while hand tracking after detection
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        # here frame is the image we get from our webcam
        ret, frame = cap.read()
        
        # flip image on the horizontal
        image = cv2.flip(frame,1)
        
        # cv2 captures image in BGR format which we want to convert into RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Set flag to prevent any changes in the image
        image.flags.writeable = False
        
        # Detection using the mediapipe model
        results = hands.process(image)
        
        # Set flag to true to make changes in the image
        image.flags.writeable = True
        
        # RGB TO BGR
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        # print(results)
        
        # rendering the results onto our image
        if(results.multi_hand_landmarks):
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
                
                # for hand detection
                print_hand_labels(image, num, hand, results)
                    
                # for angle prediction
                joint_list=[[8,5,12]]
                print_joint_angle(image, results, joint_list)

        # output the feed of our webcam to our screen
        cv2.imshow('Hand Tracking',image)
        
        # Save our image to a folder
        cv2.imwrite(os.path.join('output images', '{}.jpg'.format(uuid.uuid1())), image)
        
        # Stop when we press 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release() 
cv2.destroyAllWindows()

I0000 00:00:1706426521.058223       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


In [78]:
print(results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST])
results.multi_hand_landmarks[1].landmark[mp_hands.HandLandmark.WRIST]
# results.multi_hand_landmarks

TypeError: 'NoneType' object is not subscriptable

In [77]:
results.multi_handedness[1].classification[0]

TypeError: 'NoneType' object is not subscriptable


[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [79]:
# pyautogui.press('volumeup')
!pip install pyautogui

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 647.9 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.0/231.0 kB 812.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.5/752.5 kB 702.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 805.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

: 